## Overview of Biocopilot

**Biocopilot** is a bioinformatics application designed to streamline data analysis and visualization in biomedical research. It integrates both Python and R functionalities, providing a comprehensive tool for researchers.

### Key Dependencies

- **Python**: The application supports Python versions 3.11 and 3.12.
- **PyQt5**: Used for building the graphical user interface (GUI).
- **rpy2**: Facilitates the integration of R within Python.
- **NumPy and Pandas**: Essential libraries for numerical computations and data manipulation.
- **Jupyter**: Provides an interactive environment for running code and visualizing results.
- **Pillow and ReportLab**: Used for image processing and PDF generation.
- **R Packages**: Includes essential packages like `Seurat`, `ggplot2`, and `EnhancedVolcano`.

### Core Features

- **User-Friendly Interface**: Built with PyQt5, offering an intuitive layout for easy navigation.
- **File Management**: Supports browsing, uploading, and downloading files from local or remote servers.
- **Integrated Terminal**: Allows users to execute shell commands directly within the application.
- **AI Chat Interface**: Users can interact with an AI assistant for guidance on data analysis tasks.
- **Visualization Tools**: Provides capabilities for plotting graphs and visualizing results from both Python and R analyses.
- **Result Display Area**: Shows output from code execution, along with progress reports.

### Important Note on API Functionality

To use the AI chat interface powered by the Perplexity API, users must provide their personal API key. This key should be inserted in the following section of the code:

```python
def perplexity_request(self, query):
    api_key = ""  # Insert your Perplexity API key here
    url = "https://api.perplexity.ai/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
```
Please ensure you add your personal Perplexity API key to enable full functionality of the AI assistant feature.

In [1]:

import sys
import os
import tempfile
import re
import subprocess
import paramiko
import requests
import json
import pysftp
import time
import threading
import fitz
import csv


from PyQt5.QtGui import QPixmap, QKeyEvent, QStandardItemModel, QStandardItem, QIcon, QFont, QTextCursor, QColor, QTextCharFormat, QImage
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QDir
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QMessageBox, QVBoxLayout, QHBoxLayout, QLineEdit, QPushButton, QTextEdit, QLabel,QDialog, QTreeView, QFileSystemModel, QStyle, QAction, QMenu, QSizePolicy, QScrollArea

import rpy2.robjects as robjects
from rpy2.ipython.ggplot import image_png
from rpy2.robjects import r, globalenv
from rpy2.robjects.packages import importr, data
from rpy2.robjects import pandas2ri

from stat import S_ISDIR

from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from PyPDF2 import PdfReader, PdfWriter
from datetime import datetime
from rpy2 import robjects
import pandas as pd
from functools import partial

# Import R packages
utils = importr('utils')
base = importr('base')
lme4 = importr('lme4')
import rpy2.robjects.lib.ggplot2 as ggplot2
from rpy2.rinterface_lib.embedded import RRuntimeError

/home/iw04/.local/lib/python3.10/site-packages/rpy2/robjects/lib/ggplot2.py:70: UserWarning: This was designed againt ggplot2 versions starting with 3.3. but you have 3.5.1
  warnings.warn(


In [2]:
# Server Login dialog class
class ServerLoginBox(QDialog):
    def __init__(self):
        super().__init__()
        self.initUI()
    def initUI(self):
        self.setWindowTitle('Server Login')
        layout = QVBoxLayout()

        # Username input
        self.username_label = QLabel('User name', self)
        layout.addWidget(self.username_label)
        self.username_lineEdit = QLineEdit(self)
        layout.addWidget(self.username_lineEdit)

        # Host input
        self.host_label = QLabel('Host', self)
        layout.addWidget(self.host_label)
        self.host_lineEdit = QLineEdit(self)
        layout.addWidget(self.host_lineEdit)

        # Password input
        self.password_label = QLabel('Password', self)
        layout.addWidget(self.password_label)
        self.password_lineEdit = QLineEdit(self)
        self.password_lineEdit.setEchoMode(QLineEdit.Password)  # Hide password input
        layout.addWidget(self.password_lineEdit)

        # Submit button
        self.submit_button = QPushButton('Login', self)
        self.submit_button.clicked.connect(self.submit)
        layout.addWidget(self.submit_button)

        self.setLayout(layout)
        self.setGeometry(100, 100, 300, 200)

    def submit(self):
        self.username = self.username_lineEdit.text()
        self.host = self.host_lineEdit.text()
        self.password = self.password_lineEdit.text()
        self.accept()  # Closes the dialog and returns QDialog.Accepted


In [3]:
# Dialog for getting file names
class FileNameDialog(QDialog):
    def __init__(self, label_text, parent=None):
        super().__init__(parent)
        self.setWindowTitle('Get File Name')
        self.initUI(label_text)

    def initUI(self, label_text):
        layout = QVBoxLayout()

        self.label = QLabel(label_text, self)
        layout.addWidget(self.label)

        self.lineEdit = QLineEdit(self)
        layout.addWidget(self.lineEdit)

        self.submit_button = QPushButton('OK', self)
        self.submit_button.clicked.connect(self.accept)
        layout.addWidget(self.submit_button)

        self.setLayout(layout)
        self.setGeometry(100, 100, 300, 100)

    def get_input(self):
        return self.lineEdit.text()

In [5]:
# Custom QTextEdit for chat submission
class chatSubmitR(QTextEdit):
    def __init__(self, parent=None):
        super(chatSubmitR, self).__init__(parent)
    
    def keyPressEvent(self, event: QKeyEvent):
        if event.key() == Qt.Key_Return and (event.modifiers() & Qt.ShiftModifier):
            if hasattr(self.parent(), 'submit_chat'):
                self.parent().submit_chat()
            else:
                print("Error: Parent does not have a submit_chat method")
        else:
            super().keyPressEvent(event)

In [6]:
class CustomRoles:
    FilePathRole = Qt.UserRole + 1
    ItemTypeRole = Qt.UserRole + 2

In [7]:
# Main application class
class LLMApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.heatmap_genes = False  # Flag for heatmap gene selection
        self.R_coding_mode_status_sig = False  # Flag for R coding mode
        full_plot_path = []  # List to store plot file paths
        self.setWindowTitle("Bioinformatics Copilot 2.0")  # Set window title
        self.resize(2000, 1500)  # Set initial window size
        self.initUI()  # Initialize the user interface
        self.client = None
        self.showFiles()
        self.findPWD()
               
###______________________UI_Layout_Start______________________###
    # Initialize the user interface
    def initUI(self):
        centralWidget = QWidget()  # Create central widget
        self.setCentralWidget(centralWidget)  # Set central widget

        mainLayout = QVBoxLayout()  # Create main vertical layout

        # Directory input and upload button
        self.directoryInput = QLineEdit()  # Create input field for directory
        self.directoryInput.setPlaceholderText("Paste your file directory here...")  # Set placeholder text
        self.processButton = QPushButton("Upload")  # Create upload button
        self.processButton.clicked.connect(self.ServerOrLocal)  # Connect button to processData method

        inputLayout = QHBoxLayout()  # Create horizontal layout for input section
        inputLayout.addWidget(self.directoryInput)  # Add directory input to layout
        inputLayout.addWidget(self.processButton)  # Add upload button to layout
        mainLayout.addLayout(inputLayout)  # Add input layout to main layout

        # status bar
        self.statusBar = QLabel("Bioinformatics Copilot 2.0 is ready")  # Create status bar
        mainLayout.addWidget(self.statusBar)  # Add status bar to main layout

        chatResultLayout = QHBoxLayout()  # Create horizontal layout for chat and result area

    ##______________Chat_Layout_Start________________##
        chatLayout = QVBoxLayout()  # Create vertical layout for chat area
        self.chatDisplayBox = QTextEdit()  # Create text edit for chat display
        self.chatDisplayBox.setReadOnly(True)  # Make chat display read-only
        self.chatDisplayBox.setPlaceholderText("Copilot messages here...")  # Set placeholder text
        self.chatDisplayBox.setAcceptRichText(True)
        chatLayout.addWidget(self.chatDisplayBox)  # Add chat display to chat layout
        
        self.chatBox = QTextEdit(self)  # Create text edit for chat input
        self.chatBox.setPlaceholderText("Type to talk with Copilot...")  # Set placeholder text
        self.chatBox.setFixedHeight(100)  # Set fixed height for chat input
        self.chatBox.keyPressEvent = self.shift_enter_press

        chatInputLayout = QVBoxLayout()  # Create vertical layout for chat input
        chatButtonLayout = QHBoxLayout()  # Create horizontal layout for chat buttons
        self.R_coding_mode = QPushButton("Coding mode: OFF")  # Create button for R coding mode
        self.R_coding_mode.clicked.connect(self.R_coding_mode_status)  # Connect button to R_coding_mode_status method
        self.submitChatButton = QPushButton("Submit")  # Create submit button
        self.submitChatButton.clicked.connect(self.submitChat_LLM_test)

        chatButtonLayout.addStretch(1)  # Add stretch to push buttons to the right
        chatButtonLayout.addWidget(self.R_coding_mode)  # Add R coding mode button to button layout
        chatButtonLayout.addWidget(self.submitChatButton)  # Add submit button to button layout

        chatInputLayout.addWidget(self.chatBox)  # Add chat input to chat input layout
        chatInputLayout.addLayout(chatButtonLayout)  # Add the button layout to the chat input layout
        chatLayout.addLayout(chatInputLayout)  # Add chat input layout to main chat layout

        chatResultLayout.addLayout(chatLayout)  # Add chat layout to chat result layout

    ##______________Terminal_Layout_Start________________##
        terminalLayout = QVBoxLayout()  # Create vertical layout for terminal area
        terminalDicLayout = QHBoxLayout()
        arrow_left_icon = QApplication.style().standardIcon(QStyle.SP_ArrowLeft)  # Get the left arrow icon
        self.backButton = QPushButton()
        self.backButton.setIcon(arrow_left_icon)
        self.backButton.clicked.connect(self.go_to_parent_directory)
        terminalDicLayout.addWidget(self.backButton)
        self.terminalDic = QLineEdit(self)  # Create text edit for result display
        self.terminalDic.setPlaceholderText(' ')  # Set placeholder text
        terminalDicLayout.addWidget(self.terminalDic)

        terminalLayout.addLayout(terminalDicLayout)

        self.treeView = QTreeView()
        self.model = QStandardItemModel()
        self.treeView.setModel(self.model)
        self.treeView.clicked.connect(self.directory_clicked)
        self.treeView.setContextMenuPolicy(Qt.CustomContextMenu)
        self.treeView.customContextMenuRequested.connect(self.on_context_menu)
        terminalLayout.addWidget(self.treeView)  # Add result area to result image layout

        TerminalButtonLayout = QHBoxLayout()  # Create horizontal layout for chat buttons
        self.loginServer = QPushButton("Server Login")  # Create submit button
        self.loginServer.clicked.connect(self.toServerLogin)
        self.serverUpload = QPushButton("Upload")  # Create button for R coding mode
        self.serverDownload = QPushButton("Download")  # Create submit button

        TerminalButtonLayout.addWidget(self.loginServer)  # Add submit button to button layout
        TerminalButtonLayout.addWidget(self.serverUpload)  # Add R coding mode button to button layout
        TerminalButtonLayout.addWidget(self.serverDownload)  # Add submit button to button layout

        terminalLayout.addLayout(TerminalButtonLayout)  # Add the button layout to the chat input layout

        self.terminalWin = QTextEdit()  # Create text edit for result display
        self.terminalWin.setReadOnly(True)  # Make result area read-only
        self.terminalWin.setFont(QFont("Courier", 10))  # Set a monospaced font for better readability
        self.terminalWin.setStyleSheet("background-color: black; color: white;")  # Terminal-like colors
        terminalLayout.addWidget(self.terminalWin)  # Add image label to result image layout

        self.terminalCom = QLineEdit(self)  # Create text edit for result display
        self.terminalCom.returnPressed.connect(self.executeCommandsession)
        self.terminalCom.setPlaceholderText("Type your command here...")  # Set placeholder text
        terminalLayout.addWidget(self.terminalCom)  # Add the button layout to the chat input layout

        chatResultLayout.addLayout(terminalLayout)  # Add terminal layout to chat result layout

    ##______________Result_Layout_Start________________##
        resultImageLayout = QVBoxLayout()  # Create vertical layout for result and image area

        self.resultArea = QTextEdit()  # Create text edit for result display
        self.resultArea.setPlaceholderText("Code and progress report")  # Set placeholder text
        self.resultArea.setReadOnly(True)  # Make result area read-only
        self.resultArea.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Preferred)

        # Create a QLabel for image display
        self.imageLabel = QLabel()
        self.imageLabel.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        # Create a QScrollArea to wrap the imageLabel
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setWidget(self.imageLabel)

        # Add widgets to the layout
        resultImageLayout.addWidget(self.resultArea)  # Add result area to result image layout
        resultImageLayout.addWidget(scroll_area)  # Add scroll area to result image layout

        # Set stretch factors to adjust the size
        resultImageLayout.setStretch(0, 1)  # Smaller stretch factor for the resultArea
        resultImageLayout.setStretch(1, 3)  # Larger stretch factor for the imageLabel (via scroll_area)

        chatResultLayout.addLayout(resultImageLayout)  # Add result image layout to chat result layout

        mainLayout.addLayout(chatResultLayout)
        centralWidget.setLayout(mainLayout)  # Set main layout for central widget
###______________________UI_Layout_End______________________###

###______________________Server_Fucntions_Start______________________###
    ##______________Terminal_Interaction_Start________________##
    def executeCommandsession(self):
        command = self.terminalCom.text().strip()  # Get the command from QLineEdit
        if command:  # Ensure there is a command to run
            self.terminalWin.append(f"> {command}")  # Display the command
            if self.client is not None and self.channel is not None:
                self.channel.send(command + '\n')  # Send command
                time.sleep(1)
                self.termOutput()
            else:
                output = subprocess.run(command, capture_output=True, text=True)
                self.terminalWin.append(output.stdout)
                self.terminalCom.clear()

    def termOutput(self):
        self.term_output = ''
        while self.channel.recv_ready():
            self.term_output += self.channel.recv(4096).decode('utf-8')
            if self.term_output:
                #self.findUserName("[?2004h")
                #self.removeANSI("[?2004h")
                #print(self.term_output)
                #print(self.user_name)
                #self.terminalCom.setPlaceholderText(self.user_name)
                self.remove_ansi_escape_sequences()
                self.terminalWin.append(self.term_output_clean)
                self.terminalCom.clear()

    def findUserName(self, sequence):
        pattern = re.compile(r'\Q' + sequence + r'\E.*?\n(.*)')
        match = pattern.search(self.term_output)
        if match:
            self.user_name =  match.group(1)
        return None

    def removeANSI(self, sequence):
        ansi_escape = re.compile(r'(\x9B|\x1B\[)[0-?]*[ -/]*[@-~]')
        self.term_output = ansi_escape.sub('', self.term_output)
        pattern = re.compile(r'\Q' + sequence + r'\E.*?\n.*\n')
        self.term_output = pattern.sub('', self.term_output)
        return self.term_output 

    def remove_ansi_escape_sequences(self):
        ansi_escape_pattern = re.compile(r'(\x9B|\x1B\[)[0-?]*[ -\/]*[@-~]')
        self.term_output_clean = ansi_escape_pattern.sub('', self.term_output)

    def executeCommand(self):
        command = self.terminalCom.text().strip()  # Get the command from QLineEdit
        if command:  # Ensure there is a command to run
            self.terminalWin.append(f"> {command}")  # Display the command
            if self.client is not None:
                stdin, stdout, stderr = self.client.exec_command(command)
                output = stdout.read().decode('utf-8')
                self.terminalWin.append(output)
                self.terminalCom.clear()
            else:
                output = subprocess.run(command, capture_output=True, text=True)
                self.terminalWin.append(output.stdout)
                self.terminalCom.clear()

    ##______________Terminal_Window__Start________________##
    def go_to_parent_directory(self):
        parent_path = os.path.dirname(self.current_path)
        if parent_path != self.current_path:  # Ensure it does not go above the root directory
            self.current_path = parent_path
            self.setup_and_load_directory(parent_path)      
    
    def findPWD(self):
        if self.client is not None:
            output = self.current_path
        else:
            output = subprocess.run("pwd", capture_output=True, text=True)
            output = output.stdout.strip()
        self.terminalDic.clear()
        self.terminalDic.setPlaceholderText(output)

    def showFiles(self):
        if self.client is not None:
            stdin, stdout, stderr = self.client.exec_command("pwd")
            self.current_path = stdout.read().decode('utf-8').strip()
            self.setup_and_load_directory(self.current_path)
        else:
            self.model = QFileSystemModel()
            self.model.setRootPath(QDir.rootPath())

            # Set up the tree view for local directory browsing
            self.treeView.setModel(self.model)
            self.treeView.setRootIndex(self.model.index(QDir.rootPath()))
            # Optionally set to hide files and only show directories
            self.model.setFilter(QDir.Dirs | QDir.NoDotAndDotDot)

    def directory_clicked(self, index):
        item = self.model.itemFromIndex(index)
        path = item.data(CustomRoles.FilePathRole)
        item_type = item.data(CustomRoles.ItemTypeRole)
        if item_type == "directory":
            self.current_path = path
            self.setup_and_load_directory(path)
        elif item_type == "file":
            self.directoryInput.setText(path)
            
    def setup_and_load_directory(self, path):
        self.model = QStandardItemModel()
        stdin, stdout, stderr = self.client.exec_command(f"ls -l {path}")
        entries = stdout.read().decode().strip().split('\n')
        self.model.clear() 
        root_item = QStandardItem(os.path.basename(path) or '/')
        self.model.appendRow(root_item)
        for entry in entries:
            parts = entry.split()
            if len(parts) > 8: 
                item_name = parts[8]
                if parts[0][0] == 'd': 
                    self.folder_icon = QApplication.style().standardIcon(QStyle.SP_DirIcon)
                    item = QStandardItem(self.folder_icon, item_name)
                    item.setEditable(False)
                    item.setData(f"{path}/{item_name}", CustomRoles.FilePathRole)
                    item.setData("directory", CustomRoles.ItemTypeRole)
                    root_item.appendRow(item)
                elif parts[0][0] == '-': 
                    self.file_icon = QApplication.style().standardIcon(QStyle.SP_FileIcon)
                    item = QStandardItem(self.file_icon, item_name)
                    item.setEditable(False)
                    item.setData(f"{path}/{item_name}", CustomRoles.FilePathRole)
                    item.setData("file", CustomRoles.ItemTypeRole)
                    root_item.appendRow(item)
        self.treeView.setModel(self.model)
        self.treeView.expandAll()
    
    ##______________Context_Menu__Start________________##   
    def on_context_menu(self, point):
        index = self.treeView.indexAt(point)
        menu = QMenu()

        # Determine if the click was on a valid item
        if index.isValid():
            item = self.model.itemFromIndex(index)
            path = item.data(CustomRoles.FilePathRole)

            # Add actions for when a file or folder is clicked
            copy_path_action = QAction('Copy Path', self)
            copy_path_action.triggered.connect(lambda: self.copy_path(item))
            menu.addAction(copy_path_action)

            rename_action = QAction('Rename', self)
            rename_action.triggered.connect(lambda: self.rename_item(item))
            menu.addAction(rename_action)

            download_action = QAction('Download', self)
            download_action.triggered.connect(lambda: self.download_item(item))
            menu.addAction(download_action)
        else:
            # Use the current directory if no specific item is clicked
            path = self.current_path
            # Add actions that are always available
            new_folder_action = QAction('New Folder', self)
            new_folder_action.triggered.connect(lambda: self.create_new_folder(path))
            menu.addAction(new_folder_action)

            new_file_action = QAction('New File', self)
            new_file_action.triggered.connect(lambda: self.create_new_file(path))
            menu.addAction(new_file_action)

        menu.exec_(self.treeView.viewport().mapToGlobal(point))
        
    def executeCommand(self, command):
        self.chatDisplayBox.append("Command recieved: " + command)
        if command:
            self.terminalWin.append(f"> {command}")
            if self.client is not None:
                stdin, stdout, stderr = self.client.exec_command(command)
                output = stdout.read().decode('utf-8')
                error = stderr.read().decode('utf-8')
                if output:
                    self.terminalWin.append(output)
                if error:
                    self.terminalWin.append(f"Error: {error}")
            else:
                self.terminalWin.append("Not connected to the server.")
            self.terminalCom.clear()

    def copy_path(self, item):
        clipboard = QApplication.clipboard()
        file_path = item.data(CustomRoles.FilePathRole) 
        print(file_path) 
        clipboard.setText(file_path)
    
    def rename_item(self, item):
        path = item.data(CustomRoles.FilePathRole)
        dialog = FileNameDialog("Enter new name:", self)
        if dialog.exec_() == QDialog.Accepted:
            new_name = dialog.get_input()
            if new_name:
                new_path = os.path.join(os.path.dirname(path), new_name)
                command = f"mv {path} {new_path}"
                self.client.exec_command(command)
                self.setup_and_load_directory(os.path.dirname(path))

    def create_new_folder(self, path):
        dialog = FileNameDialog("Enter folder name:", self)
        if dialog.exec_() == QDialog.Accepted:
            folder_name = dialog.get_input()
            if folder_name:
                command = f"mkdir {os.path.join(path, folder_name)}"
                self.chatDisplayBox.append(command)
                self.client.exec_command(command)
                self.setup_and_load_directory(path)
                

    def create_new_file(self, path):
        dialog = FileNameDialog("Enter file name:", self)
        if dialog.exec_() == QDialog.Accepted:
            file_name = dialog.get_input()
            if file_name:
                command = f"touch {os.path.join(path, file_name)}"
                self.client.exec_command(command)
                self.setup_and_load_directory(path)

    def download_item(self, item):
        path = item.data(CustomRoles.FilePathRole)
        try:
            # Check if the item is a directory or a file
            with self.client.open_sftp() as sftp:
                if self.is_directory(sftp, path):
                    local_path = self.fetch_directory(path, "Biocopilot_Downloads")
                else:
                    local_path = self.fetch_file(path, "Biocopilot_Downloads")
            QMessageBox.information(self, "Download", f"Downloaded to {local_path}")
        except Exception as e:
            QMessageBox.warning(self, "Download Error", f"Failed to download: {str(e)}")

    def is_directory(self, sftp, path):
        try:
            return S_ISDIR(sftp.stat(path).st_mode)
        except IOError:
            return False

    def fetch_directory(self, remote_path, download_folder):
        with self.client.open_sftp() as sftp:
            self.app_path = os.getcwd()
            # Create a local directory with the same structure as the remote directory
            local_dir = os.path.join(self.app_path, download_folder, os.path.basename(remote_path))
            os.makedirs(local_dir, exist_ok=True)

            for entry in sftp.listdir_attr(remote_path):
                remote_file_path = os.path.join(remote_path, entry.filename)
                local_file_path = os.path.join(local_dir, entry.filename)
                if S_ISDIR(entry.st_mode):
                    # Recursively download subdirectories
                    self.fetch_directory(remote_file_path, os.path.join(download_folder, os.path.basename(remote_path)))
                else:
                    # Download files
                    sftp.get(remote_file_path, local_file_path)
                    self.resultArea.append(f"File downloaded to: {local_file_path}")

            return local_dir

    ##______________Terminal_Login__Start________________##
    def startSession(self):
        self.channel = self.client.get_transport().open_session()
        self.channel.get_pty()  # Get a pseudo-terminal
        self.channel.invoke_shell()

    def toServerLogin(self):
        UserInput = ServerLoginBox()
        self.client = MySSHClient()
        self.client.load_system_host_keys()
        if UserInput.exec_() == QDialog.Accepted:
            try:
                self.client.connect(hostname=UserInput.host, username=UserInput.username, password=UserInput.password)
            except:
                self.terminalWin.append("Login was failed.")
            else:
                self.terminalWin.append("Login was successful.")
                self.showFiles()
                self.findPWD()
                self.startSession()
                self.termOutput()   
###______________________Server_Fucntions_END______________________###

###______________________Chat_Fucntions_Start______________________###

    def shift_enter_press(self, event):
        if event.key() == Qt.Key_Return and (event.modifiers() & Qt.ShiftModifier):
            self.submit_chat()
        else:
            QTextEdit.keyPressEvent(self.chatBox, event)

    # INSERT API KEY HERE
    def perplexity_request(self, query):
        api_key = ""
        url = "https://api.perplexity.ai/chat/completions"
        
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": "mixtral-8x7b-instruct",
            "messages": [{"role": "user", "content": query}]
        }
        
        response = requests.post(url, headers=headers, data=json.dumps(data))
        
        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content']
        else:
            return f"Error: {response.status_code}, {response.text}"

    def format_response_for_chatbox(self, response):
        def escape_html(text):
            return text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
    
        def replace_code_block(match):
            code = match.group(1).strip()  # Remove leading/trailing whitespace
            escaped_code = escape_html(code)
            formatted_code = escaped_code.replace('\n', '<br>')
            return f'<pre style="background-color: #1e1e1e; color: #dcdcdc; padding: 5px; border-radius: 5px; font-family: \'Consolas\', \'Courier New\', monospace; white-space: pre-wrap; word-wrap: break-word; margin: 0;">{formatted_code}</pre>'
    
        # Use regex to find and replace code blocks
        formatted_response = re.sub(r'```([\s\S]*?)```', replace_code_block, response)
    
        # Replace newlines with <br> tags outside of code blocks
        formatted_response = formatted_response.replace('\n', '<br>')
    
        # Add "Copilot:" prefix
        formatted_response = f"<b>Copilot:</b> {formatted_response}"
    
        return formatted_response
    
    def submit_chat(self):
        query = self.chatBox.toPlainText()
        response = self.perplexity_request(query)
    
        # Format the entire Q&A pair as a single HTML string
        formatted_qa = f"""
        <hr style='border-top: 1px solid #bbb; margin: 5px 0;'>
        <p style='margin: 0;'><b>You:</b> {query}</p>
        <p style='margin: 0;'>{self.format_response_for_chatbox(response)}</p>
        """
    
        # Insert the formatted HTML into the chat box
        cursor = self.chatDisplayBox.textCursor()
        cursor.movePosition(cursor.End)
        cursor.insertHtml(formatted_qa)
        self.chatDisplayBox.setTextCursor(cursor)
        self.chatDisplayBox.ensureCursorVisible()
    
        self.chatBox.clear()
###______________________Chat_Fucntions_END______________________###

###______________________R_Fucntions_Start______________________###
    def ServerOrLocal(self):
        if self.client is not None:
            self.processData_Server()
        else:
            self.processData()
        
    def determineFunction(self, text):
        if "umap" in text:
            return "UMAP"
        elif "imagefeatureplot" in text:
            return "ImageFeaturePlot"
        elif "imagedimplot" in text:
            return "ImageDimPlot"
        elif "heatmap" in text:
            return "Heatmap"
        elif "kegg" in text:
            return "KEGG"
        elif "report" in text:
            return "Report"
    
    def submitChat_LLM_test(self):
        r = robjects.r  # Create an instance of R

        if self.R_coding_mode_status_sig == True:  # Check if R coding mode is active
            chat_text = self.chatBox.toPlainText()  # Get the full text from chat box
            command_string = self.chatBox.toPlainText().strip()  # Get and clean the text from chat box
            path_match = re.search(r'(/[^"]*\.png)', command_string)  # Search for a PNG file path

            self.chatbox_clear()  # Clear the chat box
            self.chatDisplayBox.append(f"User: {self.chatBox}")  # Add user's message to chat display
            try:
                r(f'{chat_text}')  # Execute the R code from chat text

                if path_match:  # If a PNG file path was found
                    full_plot_path = path_match.group(1)  # Extract the matched path
                    if not os.path.isfile(full_plot_path):  # Check if the file exists
                        self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")  # Display error message
                    pixmap = QPixmap(full_plot_path)  # Create a pixmap from the image file
                    scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)  # Scale the pixmap
                    self.imageLabel.setPixmap(scaled_pixmap)  # Set the scaled pixmap to the image label
                    return  # Exit the function
            except RRuntimeError as r_error:  # Catch R runtime errors
                self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")  # Display error message
            return  # Exit the function

        if self.heatmap_genes:  # Check if heatmap genes mode is active
            chat_text = self.chatBox.toPlainText().strip()  # Get and clean the text from chat box
            heatmap_genes_list = ",".join([f'"{gene}"' for gene in chat_text.split(",")])  # Create a comma-separated list of gene names

            # Extract everything after "genes:" as the gene names
            r(f'''
            heatmap_gene_list <- c({heatmap_genes_list})
            ''')  # Create an R vector of gene names
            self.Heatmap()  # Call the Heatmap method
            self.heatmap_genes = False  # Reset heatmap genes mode
            return  # Exit the function

        chat_text = self.chatBox.toPlainText().lower().strip()  # Get, lowercase, and clean the text from chat box

        self.chatbox_clear()  # Clear the chat box
        
        directory_path = self.directoryInput.text().strip()   # Get and clean the directory path
        if not directory_path:  # Check if directory path is empty
            self.chatDisplayBox.append("Copilot: Please enter a valid directory path.")  # Display error message
            return  # Exit the function
        
        self.resultArea.setText(f"loading LLM")  # Update result area with loading message
        r('library(rollama)')  # Load the rollama R library
        self.appendToResultArea(f"LLM loaded")  # Append confirmation message to result area
        
        r(f'''
        chat(
        "If the user talk about Report, output text [report].If the user talk about KEGG, output text [kegg]. If the user talk about Heatmap, output text [heatmap]. If the user talk about umap, output text [umap]. if the user talk about imagedimplot, output text [imagedimplot]. if the user talk about imagefeatureplot, output text [imagefeatureplot]. Remember that the user do not necessary say exactly the words, and you should only output the text in the [] but not other characters.",
        model = NULL,
        screen = TRUE,
        server = NULL,
        images = NULL,
        model_params = NULL,
        template = NULL
        )
        ''')
        
        r(f'''
        result_1 <- chat("{chat_text}",
        model = NULL,
        screen = TRUE,
        server = NULL,
        images = NULL,
        model_params = NULL,
        template = NULL)
        ''')
        
        LLM_result_text_1 = r(f'''
                                result_1$message$content
                              ''')
        LLM_result_text_1 = LLM_result_text_1[0] 
        LLM_result_text_1 = LLM_result_text_1.lower() 
        LLM_result_1 = self.determineFunction(LLM_result_text_1)
        
        if LLM_result_1 == "ImageDimPlot":
            if self.client is not None:
                self.ImageDimPlot_Server()
            else:
                self.ImageDimPlot()
        elif LLM_result_1 == "ImageFeaturePlot":
            if self.client is not None:
                self.ImageFeaturePlot_Server()
            else:    
                self.ImageFeaturePlot()
        elif LLM_result_1 == "UMAP":
            if self.client is not None:
                self.UMAP_Server()
            else:
                self.UMAP()       
        elif LLM_result_1 == "Heatmap":
            if self.client is not None:
                self.Heatmap_Server()
            else:
                self.Heatmap()
        elif LLM_result_1 == "KEGG":
            if self.client is not None:
                self.KEGG_Server()
            else:
                self.KEGG()
        elif LLM_result_1 == "Report":
            if self.client is not None:
                self.Report_Server()
            else:
                self.Report()
    
    def fetch_file(self, remote_path, download_folder):
        try:
            with self.client.open_sftp() as sftp:
                self.app_path = os.getcwd()
                # Ensure the local file directory exists in the application path
                local_dir = os.path.join(self.app_path, download_folder)
                os.makedirs(local_dir, exist_ok=True)

                # Determine the local path for saving the PDF
                local_path = os.path.join(local_dir, os.path.basename(remote_path))

                # Download the PDF from the remote server to the local path
                sftp.get(remote_path, local_path)
                self.resultArea.append(f"File downloaded to: {local_path}\n")
                
                return local_path
            
        except FileNotFoundError:
            self.terminalWin.append(f"Error fetching file: file not found at {remote_path}")

        except Exception as e:
            self.terminalWin.append(f"Error fetching file: {e}")
    
    def display_pdf(self, pdf_path):
        try:
            # Open the PDF document
            pdf_document = fitz.open(pdf_path)

            # Create a QWidget to hold all the pages
            container = QWidget()
            layout = QVBoxLayout(container)

            # Iterate through each page in the PDF
            for page_number in range(len(pdf_document)):
                page = pdf_document.load_page(page_number)
                pix = page.get_pixmap()
                image = QImage(pix.samples, pix.width, pix.height, pix.stride, QImage.Format_RGB888)
                pixmap = QPixmap.fromImage(image)

                # Create a QLabel for each page
                page_label = QLabel()
                page_label.setPixmap(pixmap)
                page_label.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Fixed)

                # Add the QLabel to the layout
                layout.addWidget(page_label)

            # Create a QScrollArea to enable scrolling
            scroll_area = QScrollArea()
            scroll_area.setWidgetResizable(True)
            scroll_area.setWidget(container)

            # Set the scroll area to the imageLabel area
            self.imageLabel.setLayout(QVBoxLayout())
            self.imageLabel.layout().addWidget(scroll_area)

        except Exception as e:
            self.terminalWin.append(f"Error displaying PDF: {e}")

    ##______________Server_R_Functions__Start________________##
    def processData_Server(self):
        self.rds_file_path = self.directoryInput.text().strip()
        if not self.rds_file_path:  # Check if the directory path is empty
            self.chatDisplayBox.append("Copilot: Please enter a valid directory path.")
            return
        try:
            commands = [
                'srun --partition=bmh --time=24:00:00 --mem=50G --unbuffered --pty --account=aawanggrp /bin/bash -il',
                'module load conda',
                'conda activate biocopilot',
                'R',
                'library(dplyr)',
                'library(Seurat)',
                'library(ggplot2)',
                f'TMA_data <- readRDS(\'{self.rds_file_path}\')'
            ]

            for i, command in enumerate(commands, 1):
                print(f"Sending command {i}: {command}")
                self.channel.send(command + '\n')
                command_complete = False
                while not command_complete:
                    if command_complete:
                        break
                    if self.channel.recv_ready():
                        output = self.channel.recv(1024).decode('utf-8')
                        print(f"Output of command {i}:", output)
                        if "~" or ">" in output:
                            command_complete = True
                    

                QApplication.processEvents() 
                #if errors:
                    #print("Errors from R script:", errors)
                    #self.terminalWin.append(errors)
                #else:
                    #pass  

            self.appendToResultArea(f"All packages loaded on server")
            self.Heatmap_Server()

        except Exception as e:
            print("SSH or Python error:", str(e))
            self.terminalWin.append(str(e))

    def ImageDimPlot_Server(self):
        pass

    def ImageFeaturePlot_Server(self):
        pass
    
    def UMAP_Server(self):
        pass

    def Heatmap_Server(self):
        directory_path = os.path.dirname(self.rds_file_path)
        current_time = datetime.now().strftime("%Y%m%d_%H%M")
        plot_filename = "HeatmapPlot_" + current_time
        full_plot_path = os.path.join(directory_path, plot_filename + ".pdf")
        full_csv_path = os.path.join(directory_path, plot_filename + "DEG_gene_list.csv")
        self.resultArea.append("Generating heatmap. This may take a few moments...")
        self.resultArea.update()
        
        
        try:
            commands = [
                'TMA_data.de.markers <- FindMarkers(TMA_data, ident.1 = \'PreT\', ident.2 = \'PostT\', group.by = TMA_data@meta.data$orig.ident)',
                'DEG_gene_matrix <- subset(TMA_data.de.markers, subset = avg_log2FC > 1 | avg_log2FC < -1)',
                'DEG_gene_matrix <- DEG_gene_matrix[!grepl(\'SystemControl\', rownames(DEG_gene_matrix)), ]',
                'DEG_gene_matrix <- DEG_gene_matrix[!grepl(\'Negative\', rownames(DEG_gene_matrix)), ]',
                'DEG_gene <- rownames(DEG_gene_matrix)',
                'heatmap <- DoHeatmap(TMA_data, features = DEG_gene, group.by = \'orig.ident\', slot = \'data\', hjust = 0.5, angle = 0)',
                f'ggsave(\'{full_plot_path}\', dpi = 100, plot = heatmap, limitsize = FALSE)',
                'print("heatmap done")',
                f'write.csv(DEG_gene, file = \'{full_csv_path}\', row.names = FALSE, col.names = FALSE, quote = FALSE, sep = \",\")'
            ]

            for i, command in enumerate(commands, 1):
                print(f"+Sending command {i}: {command}")
                self.channel.send(command + '\n')
                command_complete = False
                while not command_complete:
                    if command_complete:
                        break
                    if self.channel.recv_ready():
                        output = self.channel.recv(1024).decode('utf-8')
                        print(f"Output of command {i}:", output)
                        if "~" or ">" in output:
                            command_complete = True
            
            QApplication.processEvents()
            
            while True:
                stdin, stdout, stderr = self.client.exec_command(f"ls {directory_path}")
                file_list = stdout.read().decode('utf-8').strip()
                if (plot_filename + ".pdf") in file_list.split('\n'):
                    time.sleep(10)
                    print(f"{plot_filename} made")
                    break
                else:
                    time.sleep(1) 
            
            self.resultArea.append("Heatmap generation complete.")
            
            self.setup_and_load_directory(self.current_path)
            QApplication.processEvents() 
            self.resultArea.update()
            heatmap_local_path = self.fetch_file(f'{full_plot_path}', "Biocopilot_Images")
            
            
            csv_local_path = self.fetch_file(f'{full_csv_path}', "Biocopilot_Downloads")
            
            self.gene_list = []
            
            # Open the CSV file and read the first 5 entries
            with open(csv_local_path, mode='r', newline='') as file:
                csv_reader = csv.reader(file)
                for i, row in enumerate(csv_reader):
                    if 1 <= i <= 5:
                        self.gene_list.append(row[0])  # Assuming each gene is in the first column
                        
            report_path = self.Report_Server(heatmap_local_path)
            self.display_pdf(report_path)

        except Exception as e:
            print("SSH or Python error:", str(e))
            self.termOutput(str(e))       
            
    def KEGG_Server(self):
        pass
        
    def Report_Server(self, local_pdf_path):
        try:
            genes_str = ', '.join(self.gene_list)
            
            response_pdf_path = os.path.join(self.app_path, 'Biocopilot_Images', 'response.pdf')
            c = canvas.Canvas(response_pdf_path, pagesize=letter)
            width, height = letter  # Get the dimensions of the page
            
            # Set margins
            margin = 1 * inch
            text_width = width - 2 * margin
            
            # Query each gene individually and add the response to the PDF
            for gene in self.gene_list:
                prompt = (
                    f"I am a researcher conducting a study on the gene {gene} and require a detailed, up-to-date summary. "
                    "Please provide a comprehensive paragraph for each of the following categories:\n"
                    "1. Gene Function\n2. Associated Diseases\n3. Recent Research\n4. Biological Pathways\n5. Additional Information\n"
                    "Ensure that the information is accurate, credible, and based on the latest scientific literature."
                )
                api_response = self.perplexity_request(prompt)
                self.chatDisplayBox.append(api_response)
                
                # Draw the title
                c.setFont("Helvetica-Bold", 16)  # Set font to bold and larger size
                c.drawString(margin, height - margin - 20, f"Gene: {gene}")  # Title position
                
                # Set font for the body text
                c.setFont("Helvetica", 10)
                text_object = c.beginText(margin, height - margin - 50)  # Start below the title

                # Wrap text and add it to the PDF
                for line in api_response.split('\n'):
                    # Split the line into words and wrap it
                    words = line.split(' ')
                    current_line = ''
                    for word in words:
                        # Check if adding the next word exceeds the width
                        if c.stringWidth(current_line + word + ' ', 'Helvetica', 10) < text_width:
                            current_line += word + ' '
                        else:
                            text_object.textLine(current_line.strip())
                            current_line = word + ' '
                    # Add any remaining text in the line
                    if current_line:
                        text_object.textLine(current_line.strip())
                
                c.drawText(text_object)
                
                # Add a new page after each gene
                c.showPage()
            
            c.save()
            
            # Create the "Biocopilot Reports" directory
            reports_dir = os.path.join(self.app_path, 'Biocopilot Reports')
            os.makedirs(reports_dir, exist_ok=True)

            # Merge the original PDF and the response PDF
            merged_pdf_path = os.path.join(reports_dir, f"{os.path.splitext(os.path.basename(local_pdf_path))[0]}_REPORT.pdf")
            pdf_writer = PdfWriter()
            
            # Add original PDF pages
            with open(local_pdf_path, "rb") as original_pdf:
                pdf_reader = PdfReader(original_pdf)
                for page in range(len(pdf_reader.pages)):
                    pdf_writer.add_page(pdf_reader.pages[page])
            
            # Add response PDF pages
            with open(response_pdf_path, "rb") as response_pdf:
                pdf_reader = PdfReader(response_pdf)
                for page in range(len(pdf_reader.pages)):
                    pdf_writer.add_page(pdf_reader.pages[page])
            
            # Save the merged PDF in the "Biocopilot Reports" directory
            with open(merged_pdf_path, "wb") as merged_pdf:
                pdf_writer.write(merged_pdf)
            
            self.resultArea.append(f"Report PDF saved to: {merged_pdf_path}")
            
            return merged_pdf_path
            
        except Exception as e:
            self.terminalWin.append(f"Error in Report_Server: {e}")

    ##______________Local_R_Functions__Start________________##
    def R_coding_mode_status(self):
        if self.R_coding_mode_status_sig:
            self.R_coding_mode_status_sig = False  # Turn off R coding mode
            self.R_coding_mode.setText("Coding mode: OFF")  # Update button text
        else:
            self.R_coding_mode_status_sig = True  # Turn on R coding mode
            self.R_coding_mode.setText("Coding mode: ON")  # Update button text

    def processData(self):
        directory_path = self.directoryInput.text().strip()  # Get the directory path from the input field and remove leading/trailing whitespace

        if not directory_path:  # Check if the directory path is empty
            self.chatDisplayBox.append("Copilot: Please enter a valid directory path.")
            return
        
        if not os.path.exists(directory_path):  # Check if the directory exists
            self.chatDisplayBox.append(f"Copilot: Cannot find: {directory_path}")

        try:
            r = robjects.r  # Load the R script using the rpy2 library

            try:
                rds_file_path = self.directoryInput.text().strip()  # Get the RDS file path from the input field
                r(f'TMA_data <- readRDS("{rds_file_path}")')   # Load the RDS file in R
                self.resultArea.append(f'TMA_data <- readRDS("{rds_file_path}")')  # Append the R command to the result area
            except RRuntimeError as r_error:  # If there's an error loading the RDS file, display the error message
                self.chatDisplayBox.append(f"Failed to load RDS file with error: {r_error}")
           
            # If there's an error loading the RDS file, display the error message
            r('library(dplyr)')
            r('library(Seurat)')
            r('library(ggplot2)')
            r('library(EnhancedVolcano)')
            r('library(clusterProfiler)')
            r('library(org.Hs.eg.db)')
            self.appendToResultArea(f"All packages loaded")  # Inform the user that all packages have been loaded
            
            self.chatDisplayBox.append("Copilot: Data loaded successfully.")  # Inform the user that data has been loaded successfully
    
        except Exception as e:  # If any other exception occurs, display the error message
            self.chatDisplayBox.append(f"Copilot: An error occurred: {e}")
   
    def chatbox_clear(self):
        if self.chatBox.toPlainText():  # Check if there's text in the chat box
            self.chatDisplayBox.append(f"User: {self.chatBox.toPlainText()}")  # Add user's message to chat display
            self.chatBox.clear()  # Clear the chat input box

    def find_path(self):
        command_string = self.chatBox.toPlainText().strip()  # Get and clean the text from chat box
        file_path = re.findall(r'"([^"]*)"', command_string)  # Find all strings enclosed in quotes
        full_plot_path = file_path  # Assign found paths to full_plot_path
        return  
    
    def appendToResultArea(self, text):
        current_text = self.resultArea.toPlainText()  # Get current text in result area
        new_text = f"{current_text}\n{text}" if current_text else text  # Add new text with newline if there's existing text
        self.resultArea.setPlainText(new_text)  # Set updated text in result area
        
    def ImageDimPlot(self):
        self.chatDisplayBox.append(f"Copilot: OK. I am making an ImageDimPlot.")  # Inform the user that an ImageDimPlot is being created
        r = robjects.r
        try:
            # Get the directory path from the input
            rds_file_path = self.directoryInput.text().strip() 
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "ImageDimPlot.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            try:
                # Execute R code to create and save the ImageDimPlot
                r(f'''
                IDP <- ImageDimPlot(TMA_data, axes = TRUE, size = 1, combine = TRUE)
                ggsave("{full_plot_path}", IDP, width=20, height=15,dpi=600, limitsize=FALSE)
                ''')
                # Display the R code in the result area
                self.resultArea.append(f'''
                IDP <- ImageDimPlot(TMA_data, axes = TRUE, size = 1, combine = TRUE)
                ggsave("{full_plot_path}", IDP, width=20, height=15, dpi=600, limitsize=FALSE)
                ''')
            except RRuntimeError as r_error:
                # Handle R runtime errors
                self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
            # Check if the plot file was created successfully
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")
            # Display the plot in the GUI
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except Exception as e:
            self.chatDisplayBox.append(f"Copilot failed: {e}")
    
    def ImageFeaturePlot(self):
        self.chatDisplayBox.append(f"Copilot: Sure. I am making an ImageFeaturePlot.")
        r = robjects.r
        try:
            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "ImageFeaturePlot.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            try:
                r(f'''
                IFP <- ImageFeaturePlot(TMA_data, axes = TRUE, size = 1, feature = "MALAT1")
                ggsave("{full_plot_path}", IFP, dpi=600,width=20, height=15, limitsize=FALSE)
                ''')
                self.resultArea.append(f'''
                IFP <- ImageFeaturePlot(TMA_data, axes = TRUE, size = 1, feature = "MALAT1")
                ggsave("{full_plot_path}", IFP, dpi=600,width=20, height=15, limitsize=FALSE)
                ''')
            except RRuntimeError as r_error:
                self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except Exception as e:
            self.resultArea.setText(f"Copilot failed: {e}")

    def UMAP(self):
        self.chatDisplayBox.append(f"Copilot: Sure. I am making a UMAP.")
        r = robjects.r
        try:
            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "UMAP.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            try:
                r(f'''
                umap_data <- FetchData(TMA_data, vars = c("umap_1", "umap_2", "matched_clusters"))
                cluster_centers <- umap_data %>%
                    group_by(matched_clusters) %>%
                    summarise(
                        umap_1 = mean(umap_1),
                        umap_2 = mean(umap_2)
                    )     
                umap_plot <- DimPlot(TMA_data, reduction = "umap")
                UMAP <- umap_plot + 
                    geom_text(data = cluster_centers, aes(x = umap_1, y = umap_2, label = matched_clusters), 
                            inherit.aes = FALSE, nudge_x = 0.5, nudge_y = 0.5, check_overlap = TRUE)
                ggsave("{full_plot_path}", UMAP, dpi=600, limitsize=FALSE)
                ''')
                self.resultArea.append(f'''
                umap_data <- FetchData(TMA_data, vars = c("umap_1", "umap_2", "matched_clusters"))
                cluster_centers <- umap_data %>%
                    group_by(matched_clusters) %>%
                    summarise(
                        umap_1 = mean(umap_1),
                        umap_2 = mean(umap_2)
                    )     
                umap_plot <- DimPlot(TMA_data, reduction = "umap")
                UMAP <- umap_plot + 
                    geom_text(data = cluster_centers, aes(x = umap_1, y = umap_2, label = matched_clusters), 
                            inherit.aes = FALSE, nudge_x = 0.5, nudge_y = 0.5, check_overlap = TRUE)
                ggsave("{full_plot_path}", UMAP, dpi=600, limitsize=FALSE)
                ''')
            except RRuntimeError as r_error:
                self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except Exception as e:
            self.resultArea.setText(f"Copilot failed: {e}")        
    
    def Heatmap(self):
        self.chatDisplayBox.append(f"Copilot: Sure. I am making a heatmap.")
        r = robjects.r
        try:
            # Read the RDS file
            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "HeatmapPlot.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            from rpy2.robjects.vectors import StrVector
            r(f'''
            TMA_data.de.markers <- FindMarkers(TMA_data, ident.1 = "PreT", ident.2 = "PostT", group.by = TMA_data@meta.data$orig.ident)
            DEG_gene_matrix <- subset(TMA_data.de.markers, subset = avg_log2FC > 1 | avg_log2FC < -1)
            DEG_gene_matrix <- DEG_gene_matrix[!grepl("SystemControl", rownames(DEG_gene_matrix)), ]
            DEG_gene_matrix <- DEG_gene_matrix[!grepl("Negative", rownames(DEG_gene_matrix)) ,]
            DEG_gene <- rownames(DEG_gene_matrix)
              ''')
            
            r(f'''
            heatmap <- DoHeatmap(TMA_data, features = DEG_gene, group.by = "orig.ident", slot = 'data', hjust = 0.5, angle = 0)
            ggsave("{full_plot_path}", heatmap, dpi = 100, limitsize = FALSE)
            ''')
            self.resultArea.append(f'''
            heatmap <- DoHeatmap(TMA_data, features = DEG_gene, group.by = "orig.ident", slot = 'data', hjust = 0.5, angle = 0)
            ggsave("{full_plot_path}", heatmap, dpi = 100, limitsize = FALSE)
            ''')
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append("Failed to save heatmap plot.")
                return
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except RRuntimeError as e:
            self.chatDisplayBox.append(f"Failed to generate heatmap: {e}")
        
        heatmap_genes = r['DEG_gene']
        
    def KEGG(self):
        self.chatDisplayBox.append(f"Copilot: Of course. I am making a KEGG plot.")
        r = robjects.r
        try:
            # Read the RDS file
            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "KEGG.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            from rpy2.robjects.vectors import StrVector
            r(f'''
            organism <- org.Hs.eg.db
            KEGG_Genes <- rownames(TMA_data)
            ids <- bitr(KEGG_Genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb=organism)
            ids_kegg <- ids$ENTREZID
            kegg.enrichment <- enrichKEGG(ids_kegg,
            organism = "hsa",
            pvalueCutoff = 0.05,
            pAdjustMethod = "none",
            minGSSize = 10,
            maxGSSize = 500,
            qvalueCutoff = 0.2,
            use_internal_data = FALSE
            )
            dt_plot <- dotplot(kegg.enrichment, showCategory=46, font.size=12)
            ggsave("{full_plot_path}",dt_plot, width = 20, height = 20)
            ''')
            self.resultArea.append(f'''
            organism <- org.Hs.eg.db
            KEGG_Genes <- rownames(TMA_data)
            ids <- bitr(KEGG_Genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb=organism)
            ids_kegg <- ids$ENTREZID
            kegg.enrichment <- enrichKEGG(ids_kegg,
            organism = "hsa",
            pvalueCutoff = 0.05,
            pAdjustMethod = "none",
            minGSSize = 10,
            maxGSSize = 500,
            qvalueCutoff = 0.2,
            use_internal_data = FALSE
            )
            dt_plot <- dotplot(kegg.enrichment, showCategory=46, font.size=12)
            ggsave("{full_plot_path}",dt_plot, width = 20, height = 20
            ''')
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append("Failed to save KEGG plot.")
                return
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except RRuntimeError as e:
            self.chatDisplayBox.append(f"Failed to generate KEGG: {e}")

    def Report(self):
        r = robjects.r
        self.chatDisplayBox.append(f"Copilot: Of course. I am making a Report.")
        self.directoryInput = self.directoryInput.text().strip()
        self.directoryInput = os.path.dirname(self.directoryInput)

        # Get the current time and format it
        current_time = datetime.now().strftime("%Y%m%d_%H%M")
        directory_path = os.path.join(self.directoryInput, f"Report_{current_time}")
        os.makedirs(directory_path, exist_ok=True)

        png_files = []
        
        # Generate ImageDimPlot
        plot_filename = "ImageDimPlot.png"
        full_plot_path = os.path.join(directory_path, plot_filename)
        png_files.append(full_plot_path)
        try:
            r(f'''
            IDP <- ImageDimPlot(TMA_data, axes = TRUE, size = 1, combine = TRUE)
            ggsave("{full_plot_path}", IDP, width=20, height=15,dpi=600, limitsize=FALSE)
            ''')
        except rpy2.rinterface_lib.embedded.RRuntimeError as r_error:
            self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
        if not os.path.isfile(full_plot_path):
            self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")

        # Generate ImageFeaturePlot
        plot_filename = "ImageFeaturePlot.png"
        full_plot_path = os.path.join(directory_path, plot_filename)
        png_files.append(full_plot_path)
        try:
            r(f'''
            IFP <- ImageFeaturePlot(TMA_data, axes = TRUE, size = 1, feature = "MALAT1")
            ggsave("{full_plot_path}", IFP, dpi=600,width=20, height=15, limitsize=FALSE)
            ''')
        except rpy2.rinterface_lib.embedded.RRuntimeError as r_error:
            self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
        if not os.path.isfile(full_plot_path):
            self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")

        # Generate UMAP Plot
        plot_filename = "UMAP.png"
        full_plot_path = os.path.join(directory_path, plot_filename)
        png_files.append(full_plot_path)
        try:
            r(f'''
            umap_data <- FetchData(TMA_data, vars = c("umap_1", "umap_2", "matched_clusters"))
            cluster_centers <- umap_data %>%
                group_by(matched_clusters) %>%
                summarise(
                    umap_1 = mean(umap_1),
                    umap_2 = mean(umap_2)
                )     
            umap_plot <- DimPlot(TMA_data, reduction = "umap")
            UMAP <- umap_plot + 
                geom_text(data = cluster_centers, aes(x = umap_1, y = umap_2, label = matched_clusters), 
                        inherit.aes = FALSE, nudge_x = 0.5, nudge_y = 0.5, check_overlap = TRUE)
            ggsave("{full_plot_path}", UMAP, dpi=600, limitsize=FALSE)
            ''')
        except rpy2.rinterface_lib.embedded.RRuntimeError as r_error:
            self.chatDisplayBox.append(f"Failed to generate the plot with error: {r_error}")
        if not os.path.isfile(full_plot_path):
            self.chatDisplayBox.append(f"Copilot cannot save plot at {full_plot_path}")

        # Generate HeatmapPlot
        plot_filename = "HeatmapPlot.png"
        full_plot_path = os.path.join(directory_path, plot_filename)
        png_files.append(full_plot_path)
        r(f'''
        TMA_data.de.markers <- FindMarkers(TMA_data, ident.1 = "PreT", ident.2 = "PostT", group.by = TMA_data@meta.data$orig.ident)
        DEG_gene_matrix <- subset(TMA_data.de.markers, subset = avg_log2FC > 1 | avg_log2FC < -1)
        DEG_gene_matrix <- DEG_gene_matrix[!grepl("SystemControl", rownames(DEG_gene_matrix)), ]
        DEG_gene_matrix <- DEG_gene_matrix[!grepl("Negative", rownames(DEG_gene_matrix)) ,]
        DEG_gene <- rownames(DEG_gene_matrix)
        ''')
        r(f'''
        heatmap <- DoHeatmap(TMA_data, features = DEG_gene, group.by = "orig.ident", slot = 'data', hjust = 0.5, angle = 0)
        ggsave("{full_plot_path}", heatmap, dpi = 100, limitsize = FALSE)
        ''')
        if not os.path.isfile(full_plot_path):
            self.chatDisplayBox.append("Failed to save heatmap plot.")

        # Generate KEGG Plot
        plot_filename = "KEGG.png"
        full_plot_path = os.path.join(directory_path, plot_filename)
        png_files.append(full_plot_path)
        r(f'''
        organism <- org.Hs.eg.db
        KEGG_Genes <- rownames(TMA_data)
        ids <- bitr(KEGG_Genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb=organism)
        ids_kegg <- ids$ENTREZID
        kegg.enrichment <- enrichKEGG(ids_kegg,
                                    organism = "hsa",
                                    pvalueCutoff = 0.05,
                                    pAdjustMethod = "none",
                                    minGSSize = 10,
                                    maxGSSize = 500,
                                    qvalueCutoff = 0.2,
                                    use_internal_data = FALSE
                                    )
        dt_plot <- dotplot(kegg.enrichment, showCategory=46, font.size=12)
        ggsave("{full_plot_path}",dt_plot, width = 20, height = 20)
        ''')
        if not os.path.isfile(full_plot_path):
            self.chatDisplayBox.append("Failed to save KEGG plot.")

        # Combine PNG files into a single PDF
        output_pdf = os.path.join(directory_path, "combined_report.pdf")
        
        images = [Image.open(png).convert('RGB') for png in png_files if os.path.isfile(png)]
        if images:
            images[0].save(output_pdf, save_all=True, append_images=images[1:])
            self.chatDisplayBox.append(f"Combined PDF saved as {output_pdf}")
        else:
            self.chatDisplayBox.append("No images found to combine into a PDF.")
    
    def Old_Heatmap(self):
        #####This is the old heatmap version
        r = robjects.r
        try:
            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "HeatmapPlot.png"
            full_plot_path = os.path.join(directory_path, plot_filename)
            from rpy2.robjects.vectors import StrVector
            r(f'''
            DoHeatmap(TMA_data) 
            save(ht, file = "{full_plot_path}")
            ''')
            if not os.path.isfile(full_plot_path):
                self.chatDisplayBox.append("Failed to save heatmap plot.")
                return
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
        except RRuntimeError as e:
            self.chatDisplayBox.append(f"Failed to generate heatmap: {e}")
    
    def displayRPlot(self):
        r = robjects.r

        try:
            self.appendToResultArea(f"loading Seurat")
            r('library(Seurat)')
            self.appendToResultArea(f"loading ggplot2")
            r('library(ggplot2)')
            self.appendToResultArea(f"loading dplyr")
            r('library(dplyr)')
            self.appendToResultArea(f"All packages loaded")
        #except RRuntimeError as r_error:
            #self.appendToResultArea(f"Failed to load packages: {r_error}")

            ###test step 0
            self.appendToResultArea(f"test step 0")

            rds_file_path = self.directoryInput.text().strip()
            directory_path = os.path.dirname(rds_file_path)
            plot_filename = "TMA_data_plot.png"
            full_plot_path = os.path.join(directory_path, plot_filename)

            ###test step 1
            self.appendToResultArea(f"test step 1")

            # Check if the RDS path is correct
            if not os.path.exists(rds_file_path):
                self.appendToResultArea(f"Bioinformatics Copilot cannot find: {rds_file_path}")

            ###test step 2
            self.appendToResultArea(f"test step 2")

            # R code to generate and save the plot
            try:
                self.appendToResultArea(f"Bioinformatics Copilot is opening the file: {rds_file_path}")
                r(f'TMA_data <- readRDS("{rds_file_path}")')
                self.appendToResultArea(f"Bioinformatics Copilot opened the file")
            except RRuntimeError as r_error:
                self.appendToResultArea(f"Failed to load RDS file with error: {r_error}")

            try:
                r(f'''
                IDP <- ImageDimPlot(TMA_data, axes = TRUE, size = 0.5, combine = TRUE)
                ggsave("{full_plot_path}", IDP, dpi=600, limitsize=FALSE)
                ''')
            except RRuntimeError as r_error:
                self.appendToResultArea(f"Failed to generate the plot with error: {r_error}")


            self.appendToResultArea(f"Bioinformatics Copilot will save plot at {full_plot_path}")

            if not os.path.isfile(full_plot_path):
                self.appendToResultArea(f"Bioinformatics Copilot cannot save plot at {full_plot_path}")

            self.appendToResultArea(f"Bioinformatics Copilot save plot as: {full_plot_path}")

            # Load the image and display it in the QLabel
            pixmap = QPixmap(full_plot_path)
            scaled_pixmap = pixmap.scaled(self.imageLabel.size(), Qt.AspectRatioMode.KeepAspectRatio)
            self.imageLabel.setPixmap(scaled_pixmap)
            self.appendToResultArea("Bioinformatics Copilot is so good at drawing")

        except Exception as e:
            self.resultArea.setText(f"Bioinformatics Copilot LLaMA failed: {e}")
###______________________R_Fucntions_END______________________###

def main():
    try:
        app = QApplication(sys.argv)
        mainWindow = LLMApp()
        mainWindow.show()
        sys.exit(app.exec())
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [ ]:
if __name__ == "__main__":
    main()